### IMPORTS

In [1]:
import json
import subprocess
import sys
import time

In [2]:
import hashlib
import base64

In [3]:
import pkg_resources.py2_warn

In [4]:
from web3 import Web3

### LOGGING BASELINE

In [5]:
def log(msg):
    print('[LAUNCHER] ' + msg)

### LOAD DATA FROM EXTERNAL FILES

In [6]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [7]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [8]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

In [9]:
log('FETCHED DATA FROM EXTERNAL FILES')

[LAUNCHER] FETCHED DATA FROM EXTERNAL FILES


### GATEWAY CONNECTION CHECK

In [10]:
def connection(protocol):
    
    # GATEWAY TO PROTOCOL
    gateway = Web3(Web3.WebsocketProvider(
        'ws://' + settings['gateways'][protocol]['host'] + ':' + str(settings['gateways'][protocol]['port'])
    ))
    
    # IF CONNECTION IS ESTABLISHED
    if gateway.isConnected():
        
        # SHOW MESSAGE
        message = protocol + ' GATEWAY CONNECTION ESTABLISHED'
        log(message.upper())
        
        # FINALLY RETURN CONNECTION STRING
        return gateway
        
    # OTHERWISE, SHOW ERROR & DIE
    else:
        log('COULD NOT CONNECT TO GATEWAY. SHUTTING DOWN.')
        exit

### ATTEMPT TO CONNECT

In [11]:
blockchain = connection('blockchain')

[LAUNCHER] BLOCKCHAIN GATEWAY CONNECTION ESTABLISHED


In [12]:
whisper = connection('whisper')

[LAUNCHER] WHISPER GATEWAY CONNECTION ESTABLISHED


### EXTRACT THE SHH API

In [13]:
shh = whisper.geth.shh

### GENERATE WHISPER ID

In [14]:
whisper_id = shh.newKeyPair()

In [15]:
log('WHISPER ID GENERATED')

[LAUNCHER] WHISPER ID GENERATED


### HASH DEVICE IDENTIFIER

In [16]:
def hash_id(data):
    
    # REMOVE WHITESPACES
    to_string = json.dumps(data, sort_keys=False, indent=2)
    
    # ENCODE THE STRING WITH UTF8
    encoded = to_string.encode('utf-8')
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

### DEVICE OUTLINE

In [17]:
class create_device():
    def __init__(self):
        self.hash = hash_id(device_info)
    
    # LOCATE & SET DEVICE CONTRACT ADDRESS
    def set_contract(self, contract):
        self.contract = contract
    
    # REDIRECT TO PARENT
    def read(self, details):
        return self.contract.read(details)
    
    # REDIRECT TO PARENT
    def write(self, details):
        return self.contract.write(details)
    
    # REDIRECT TO PARENT
    def event(self, name):
        return self.contract.event(name)

In [18]:
device = create_device()

In [19]:
log('DEVICE ID HASHED')

[LAUNCHER] DEVICE ID HASHED


### UNIFORM SMART CONTRACT OUTLINE

In [20]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = blockchain.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': settings['keys']['public'],
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = blockchain.eth.estimateGas(tx)
            tx['gasPrice'] = blockchain.toWei(20, 'gwei')
            tx['nonce'] = blockchain.eth.getTransactionCount(settings['keys']['public'])

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = blockchain.eth.account.sign_transaction(tx,
                private_key = settings['keys']['private']
            )

            # SEND THE TRANSACTION
            blockchain.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return True
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### SERIALIZE MANAGER CONTRACTS

In [21]:
device_manager = contract(latest['devicemanager'])

In [22]:
task_manager = contract(latest['taskmanager'])

In [23]:
log('MANAGER CONTRACTS SERIALIZED')

[LAUNCHER] MANAGER CONTRACTS SERIALIZED


### SERIALIZE DEVICE CONTRACT

In [24]:
temp_contract = contract({
    'address': device_manager.read({
        'func': 'fetch_device',
        'params': device.hash
    }),
    'abi': latest['device']['abi']
})

In [25]:
device.set_contract(temp_contract)

In [26]:
log('DEVICE CONTRACT SERIALIZED')

[LAUNCHER] DEVICE CONTRACT SERIALIZED


### HELPER FUNCS

In [27]:
def filter_backlog(data):
    
    # FILTER ZEROS
    filtered = filter(lambda x: x != '0x0000000000000000000000000000000000000000', data)
    
    # CONVERT TO LIST & RETURN
    return list(filtered)

In [28]:
def parse_config(compressed):
    
    # EXPAND TO STRING
    to_bytes = base64.b64decode(compressed)
    
    # PARSE AS JSON & RETURN
    return json.loads(to_bytes)

### GLOBAL TASK BACKLOG

In [29]:
raw = device.read('details')[1]

In [30]:
backlog = filter_backlog(raw)

In [31]:
log('TASKS IN BACKLOG: ' + str(len(backlog)))

[LAUNCHER] TASKS IN BACKLOG: 3


### GLOBAL ACTIVE STATUS 

In [32]:
active = device.read('active')

In [33]:
log('CURRENT ACTIVE STATUS: ' + str(active))

[LAUNCHER] CURRENT ACTIVE STATUS: True


### GLOBAL DISCOVERY STATUS

In [34]:
discoverable = device.read('discoverable')

In [35]:
log('CURRENT DISCOVERABLE STATUS: ' + str(discoverable))

[LAUNCHER] CURRENT DISCOVERABLE STATUS: True


### GLOBAL DISCOVERY CONFIG

In [36]:
compressed = device.read('tags')

In [37]:
discovery_config = parse_config(compressed)

In [38]:
log('DISCOVERY CONFIG FETCHED')

[LAUNCHER] DISCOVERY CONFIG FETCHED


### EVENT FUNCTIONS

In [39]:
def update_details(event):
    
    # FETCH GLOBAL VARS
    global active
    global discoverable
    global backlog
    global discovery_config
    
    # EXTRACT RELEVANT VALUES
    latest_active = event['args']['active']
    latest_discoverable = event['args']['discoverable']
    latest_config = parse_config(event['args']['tags'])

    # IF ACTIVE STATUS HAS CHANGED
    if (latest_active != active):
        
        # UPDATE ACTIVE STATUS
        active = latest_active
        
        # SEND MSG
        log('ACTIVE STATUS CHANGED TO: ' + str(latest_active))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_discoverable != discoverable):
        
        # UPDATE ACTIVE STATUS
        discoverable = latest_discoverable
        
        # SEND MSG
        log('DISCOVERABLE STATUS CHANGED TO: ' + str(latest_discoverable))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_config != discovery_config):
        
        # UPDATE ACTIVE STATUS
        discovery_config = latest_config
        
        # SEND MSG
        log('DISCOVERY CONFIG CHANGED')
        
    # UPDATE BACKLOG
    raw_backlog = event['args']['backlog']
    backlog = filter_backlog(raw_backlog)

In [40]:
def update_middleware():
    
    # PRINT REACTION
    log('MIDDLEWARE UPDATE TRIGGERED')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

In [41]:
def perform_task(task):
    
    # SHOW MSG
    log('STARTING TASK: ' + task)

    # TASK RETURN PARAMS
    ipfs = 'QmWATWQ7fVPP2EFGu71UkfnqhYXDYH566qy47CnJDgvs8u'
    key = '0x4f7a87EE7A53ae8606e80FE96a47038DF8ab7956'

    # SLEEP FOR 5 SECONDS
    time.sleep(5)
    
    
    
    

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, ipfs, key]
    })

    # SHOW MSG
    log('TASK COMPLETED')

In [42]:
def process_message(event):

    # CONVERT MESSAGE PARAMS
    author = blockchain.toHex(event['sig'])
    nickname = author[0:4] + '...' + author[-4:]
    message = blockchain.toText(event['payload'])

    # CHECK THE FIRST WORD FOR TRIGGER
    first = message.split(' ')[0].lower()

    # CHAT QUERY STRING
    # !find service=nlp & location=helsinki
    
    # IF A REQUEST IS FOUND
    if (first == '!find'):

        # NUKE GARBAGE & SPLIT OUT INDIVIDUAL PARAMS
        checks = message[8:].replace(' ', '').split('&')

        # CHECK RESULTS
        results = []

        # LOOP THROUGH
        for pair in checks:

            # DECONSTRUCT KEY/VALUE
            key, value = pair.split('=')

            # MAKE SURE THE KEY EXISTS
            if (key in discovery_config):

                # CHECK & PUSH PARAM COMPARISON RESULT
                result = discovery_config[key] == value
                results.append(result)

            # OTHERWISE, DEFAULT TO BAD RESULT
            else:
                results.append(False)

        # IF ALL THE CHECKS MATCH
        if (results.count(False) == 0):
            
            # SHOW MSG
            log('DISCOVERY REQUEST DETECTED')

            # RESPONSE TEXT
            response = '@' + nickname + ' - I am available at ' + device.hash

            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['topic']['key'],
                'payload': blockchain.toHex(text=response),
                'topic': blockchain.toHex(text=settings['whisper']['topic']['name']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### START LISTENING FOR EVENTS

In [43]:
log('AWAITING EVENTS...\n')

[LAUNCHER] AWAITING EVENTS...



### CONTRACT EVENTS

In [44]:
update_event = device.event('middleware')

In [45]:
changes_event = device.event('changes')

In [46]:
assignment_event = device.event('assignment')

In [47]:
message_event = shh.newMessageFilter({
    'topic': blockchain.toHex(text=settings['whisper']['topic']['name']),
    'symKeyID': settings['whisper']['topic']['key']
})

### EVENT LOOP

In [48]:
try:
    while(True):
    
        # FETCH THE GLOBAL PARAMS
        global backlog
        global active
        global discoverable

        # ACTIVE STATUS EVENT
        for event in changes_event.get_new_entries():
            update_details(event)

        # UPDATE MIDDLEWARE EVENT
        for event in update_event.get_new_entries():
            update_middleware()

        # IF THE DEVICE IS SET TO ACTIVE
        if (active):

            # PERFORM TASKS IN BACKLOG
            for task in backlog:
                perform_task(task)
        
        # IF THE DEVICE IS DISCOVERABLE
        if (discoverable):
            
            # TRACK WHISPER REQUESTS
            for event in shh.getMessages(message_event):
                process_message(event)

# WHEN THE PROCESS IS KILLED...
except KeyboardInterrupt:
    print('\nThe process was manually stopped...')
    pass

[LAUNCHER] STARTING TASK: 0x6c2bdBdCc03852C23FB0964be23Dd192e4215512
[LAUNCHER] TASK COMPLETED
[LAUNCHER] STARTING TASK: 0x68104c1C6171f44008Aeb17Cf1E2AeD4cAE08BE5
[LAUNCHER] TASK COMPLETED
[LAUNCHER] STARTING TASK: 0x0C474E0E28AC6df46A17575CFB6175419D5a4A81
[LAUNCHER] TASK COMPLETED
[LAUNCHER] MIDDLEWARE UPDATE TRIGGERED
[LAUNCHER] DISCOVERY CONFIG CHANGED

The process was manually stopped...
